This code is a test of the effect of the SnowNLP model and is divided into two parts, Part1 and Part2.Part1 is the result of the test using SnowNLP directly, and Part2 is the result of the test after fine-tuning using the labeled dataset. In order to ensure a consistent standard of comparison, the random number seed when cutting the dataset is set to 0 and the threshold is tuned to 0.1.

In [ ]:
!pip install snownlp
!pip install pandas
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760944 sha256=8f2ef579a31521a3f057e757f555d1010851bc79695c2984187a413a666b1398
  Stored in directory: /root/.cache/pip/wheels/43/f3/70/8990fc249efeb396007766676706f71dd3d1ca3c023ce522ce
Successfully built snownlp


#Part1

In [ ]:
import pandas as pd
from snownlp import SnowNLP
from sklearn.model_selection import train_test_split

file_name = "/content/labelnews.xlsx"
df = pd.read_excel(file_name)

train_data, test_data = train_test_split(df, test_size=0.3, random_state=0)

correct = 0
total = len(test_data)
for index, row in test_data.iterrows():
    text = row['NewsContent']
    label = row['label']
    prediction = SnowNLP(text).sentiments
    predicted_label = 1 if prediction > 0.1 else 0
    if predicted_label == label:
        correct += 1

accuracy = correct / total
print(f'Accuracy: {accuracy}')


Accuracy: 0.7982456140350878


#Part2

In [ ]:
import pandas as pd
from snownlp import sentiment
from sklearn.model_selection import train_test_split

file_name = "/content/labelnews.xlsx"
df = pd.read_excel(file_name)

train_data, test_data = train_test_split(df, test_size=0.3, random_state=0)

# Extracting text and tags from the training set and fine-tuning with the training set
train_texts = train_data['NewsContent'].tolist()
train_labels = train_data['label'].tolist()

with open('positive.txt', 'w', encoding='utf-8') as pos_file, \
     open('negative.txt', 'w', encoding='utf-8') as neg_file:
    for text, label in zip(train_texts, train_labels):
        if label == 1:
            pos_file.write(text + '\n')
        else:
            neg_file.write(text + '\n')

sentiment.train('negative.txt', 'positive.txt')
sentiment.save('sentiment.marshal')

In [ ]:
# Using the trained model
sentiment.load('sentiment.marshal')

correct = 0
total = len(test_data)
for index, row in test_data.iterrows():
    text = row['NewsContent']
    label = row['label']
    prediction = SnowNLP(text).sentiments
    predicted_label = 1 if prediction > 0.1 else 0
    if predicted_label == label:
        correct += 1

accuracy = correct / total
print(f'Accuracy: {accuracy}')

Accuracy: 0.24561403508771928
